In [1]:
import f3dasm
import numpy as np
import time
from multiprocessing import Pool

In [2]:
dim = 2
iterations = 50
realizations = 10
bounds = np.tile([-1.0,1.0], (dim, 1))
#hyperparameters={'learning_rate': 2e-4}
hyperparameters={}

design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensionality=dim)

function = f3dasm.functions.Ackley(dimensionality=dim, noise=False, scale_bounds=bounds)

data = f3dasm.Data(design=design)
optimizer = f3dasm.optimization.DualAnnealing(data=data, hyperparameters=hyperparameters)
sampler = f3dasm.sampling.LatinHypercubeSampling(design=design)


# all_data = f3dasm.run_multiple_realizations(optimizer=optimizer, 
# function=function,
# sampler=sampler,
# iterations=iterations,
# realizations=realizations
# )



In [3]:
optimizer.parameter

DualAnnealing_Parameters(population=1, force_bounds=True, initial_temp=5230.0, restart_temp_ratio=2e-05, visit=2.62, accept=-5.0, no_local_search=False)

In [4]:
args = {
'optimizer': optimizer, 
'function': function,
'sampler': sampler,
'iterations': iterations,
'seed': 1


}

f3dasm.run_optimization(**args)

Data(design=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=       input               output
          x0        x1          y
0  -0.165956  0.440649  21.687648
1   0.718509 -0.674886  20.512654
2   0.714344 -0.672450  19.944225
3  -0.753425 -0.358486  14.759652
4  -0.536873 -0.429770   2.579928
5  -0.536873 -0.429770   2.579928
6  -0.536873 -0.429770   2.579928
7  -0.536873 -0.429770   2.579928
8  -0.536873 -0.429770   2.579928
9  -0.536873 -0.429770   2.579928
10 -0.536873 -0.429770   2.579928
11 -0.536873 -0.429770   2.579928
12 -0.536873 -0.429770   2.579928
13 -0.536873 -0.429770   2.579928
14 -0.536873 -0.429770   2.579928
15 -0.536873 -0.429770   2.579928
16 -0.536873 -0.429770   2.579928
17 -0.536873 -0.429770   2.579928
18 -0.536873 -0.429

In [5]:
args = {
'realizations': realizations,
'optimizer': optimizer, 
'function': function,
'sampler': sampler,
'iterations': iterations,


}
f3dasm.run_multiple_realizations(**args)

[Data(design=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=       input               output
           x0        x1          y
 0   0.807671  0.818814  20.489867
 1  -0.307864 -0.296721  16.767027
 2  -0.536873 -0.549301   6.882583
 3  -0.563581 -0.430371   3.780106
 4  -0.566428 -0.429272   3.574452
 5  -0.566428 -0.429272   3.574452
 6  -0.566428 -0.429272   3.574452
 7  -0.566428 -0.429272   3.574452
 8  -0.566428 -0.429272   3.574452
 9  -0.566428 -0.429272   3.574452
 10 -0.566428 -0.429272   3.574452
 11 -0.566428 -0.429272   3.574452
 12 -0.566428 -0.429272   3.574452
 13 -0.566428 -0.429272   3.574452
 14 -0.566428 -0.429272   3.574452
 15 -0.566428 -0.429272   3.574452
 16 -0.566428 -0.429272   3.574452
 17 -0.566428 -0.429272   3.574452

In [6]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

seed = np.random.randint(low=0, high=1e5)
all_data = []

for _ in range(realizations):
    data = f3dasm.run_optimization(
        optimizer=optimizer, function=function, sampler=sampler, iterations=iterations, seed=seed
    )
    all_data.append(data)

    # Increase seed
    seed += 1

end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 1.13s total


In [7]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]


with Pool() as pool:
    results = pool.starmap(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 0.38s total


In [8]:
results

[Data(design=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=       input                  output
           x0        x1             y
 0   0.305486  0.795750  2.171476e+01
 1  -0.810048 -0.319784  1.687853e+01
 2  -0.352350 -0.533878  1.377749e+01
 3  -0.566428 -0.429272  3.574452e+00
 4  -0.535462 -0.449487  2.216337e+00
 5  -0.536873 -0.458828  6.256329e-07
 6  -0.536873 -0.458828  6.256329e-07
 7  -0.536873 -0.458828  6.256329e-07
 8  -0.536873 -0.458828  6.256329e-07
 9  -0.536873 -0.458828  6.256329e-07
 10 -0.536873 -0.458828  6.256329e-07
 11 -0.536873 -0.458828  6.256329e-07
 12 -0.536873 -0.458828  6.256329e-07
 13 -0.536873 -0.458828  6.256329e-07
 14 -0.536873 -0.458828  6.256329e-07
 15 -0.536873 -0.458828  6.256329e-07
 16 -0.536873 -

In [9]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]


with Pool() as pool:
    results = pool.starmap_async(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 0.06s total


In [10]:
results

In [11]:
func.dfdx(np.array([0.5, 0.8]))

NameError: name 'func' is not defined

In [ ]:
if dim == 2:
    fig, ax = func.plot_data(all_data[0], px=100, domain=bounds)
    ax.scatter(func.get_global_minimum(dim)[0][0],func.get_global_minimum(dim)[0][1], s=40, c='r')
    #ax.scatter(-3.19468541,0.89682456, s=40, c='g')
    func.plot(px=100, domain=bounds)

In [ ]:
y = np.array([[0.2,0.1],[0.3,0.4]])
y + np.abs(func.get_global_minimum(2)[1])

In [ ]:
all_data[0].data['input']

In [ ]:
func.scale_bounds, func.input_domain

In [ ]:
import pandas as pd
mean_y = pd.concat([d.get_output_data() for d in all_data], axis=1).mean(axis=1)
std_y = pd.concat([d.get_output_data() for d in all_data], axis=1).std(axis=1)
mean_y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.figure(), plt.axes()
ax.plot(mean_y)
ax.fill_between(np.arange(len(mean_y)), mean_y-1.96*std_y, mean_y+1.96*std_y, color='b', alpha=.1)
#ax.set_yscale('log')
fig.show()

In [ ]:
func.get_global_minimum(2)

In [ ]:
func.descale_input(np.array([0.0,0.0]))
x = np.array([[0.0,0.0]])

In [ ]:
func.input_domain

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0]) * x + func.input_domain[:, 0]

In [ ]:
(x - func.input_domain[:, 0]) / (func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
x

In [ ]:
func.input_domain[:, 0]

In [ ]:
x - func.input_domain[:, 0]

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
o = np.array([[0.9,0.3]])
o

In [ ]:
func.scale_input(func.descale_input(o))

In [ ]:
func.descale_input(func.scale_input(o))
o

In [ ]:
scale_bounds = [-3., 1.]

In [ ]:
o = np.random.uniform(low=scale_bounds[0], high=scale_bounds[1], size=(1, func.dimensionality))
x = o
x

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
(x - func.scale_bounds[:, 0]) / (func.scale_bounds[:, 1] - func.scale_bounds[0])

In [ ]:
func.offset

In [ ]:
func.get_global_minimum(2)[0]

In [ ]:
(func.scale_bounds[:, 1] - func.scale_bounds[0])